#  LLM に知識を与える

In [7]:
pip install langchain

In [8]:
pip install langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [29]:
# OpenAI API キーの設定
import getpass
import os
api_key = getpass.getpass("OpenAI API キーを入力してください: ")
os.environ["OPENAI_API_KEY"] = api_key

OpenAI API キーを入力してください:  ········


In [1]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

model = ChatOpenAI(model="gpt-4o-mini")
result = model.invoke([HumanMessage(content="熊童子について教えて\
ください。")])
print(result.content)

熊童子（くまどうじ）は、日本に伝わる民間信仰や伝説に登場する妖怪や神霊の一つです。主に東北地方や山形県などで語られる存在で、特に「熊童子」はその姿や性格に地域ごとのバリエーションがあります。

### 特徴
- **外見**: 熊童子は一般的に小さなクマのような姿を持ち、体毛がもふもふしています。手足の形状に特徴があり、爪が鋭いとされることがあります。
- **性格**: 人間に対して友好的な性格とされることが多いですが、時にはいたずらをすることもあると伝えられています。

### 伝説や信仰
- 地域によっては、熊童子は山の神や守り神として崇められることがあり、特に自然や動物の守護者として信じられています。
- 特定の場所や祭りで祭られることもあります。

### 考察
熊童子は、日本の伝承や神話における動物の神格化の一例であり、自然と人間の関係を表す象徴とも言えます。こうした存在は、地域の文化や伝統を理解する上で重要な要素となります。

熊童子についての情報は、地域や伝承によって異なるため、詳細を知りたい場合は特定の地域の伝説や民話を調べると良いでしょう。


In [12]:
from langchain_core.prompts import ChatPromptTemplate

#1 プロンプトテンプレートの作成
message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

model = ChatOpenAI(model="gpt-4o-mini")
chain = prompt | model

question_text = " 熊童子について教えてください。"
information_text = """\
熊童子はベンケイソウ科コチレドン属の多肉植物です。
葉に丸みや厚みがあり、先端には爪のような突起があることから「熊の手」という\
愛称で人気を集めています。
花はオレンジ色のベル型の花を咲かせることがあります。"""

response = chain.invoke({"context": information_text, "question":
question_text})
print(response.content)

熊童子はベンケイソウ科コチレドン属の多肉植物で、葉に丸みや厚みがあり、先端に爪のような突起があるため「熊の手」という愛称があります。また、オレンジ色のベル型の花を咲かせることもあります。


# 文書の構造化

In [14]:
from langchain_core.documents import Document

#1 Documentクラスオブジェクトの作成
document = Document(
        page_content="""\
セダムはベンケイソウ科マンネングザ属で、日本にも自生しているポピュラーな多\
肉植物です。
種類が多くて葉の大きさや形状、カラーバリエーションも豊富なので、組み合わせ\
て寄せ植えにしたり、庭のグランドカバーにしたりして楽しむことができます。
とても丈夫で育てやすく、多肉植物を初めて育てる方にもおすすめです。""",
        metadata={"source": "succulent-plants-doc"},
    )

print(document)

page_content='セダムはベンケイソウ科マンネングザ属で、日本にも自生しているポピュラーな多肉植物です。
種類が多くて葉の大きさや形状、カラーバリエーションも豊富なので、組み合わせて寄せ植えにしたり、庭のグランドカバーにしたりして楽しむことができます。
とても丈夫で育てやすく、多肉植物を初めて育てる方にもおすすめです。' metadata={'source': 'succulent-plants-doc'}


# langchain_chromaの利用

In [18]:
pip install langchain_chroma

  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
Using cached tenacity-9.1.2-py3-none-any.whl (28 kB)
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.2.2
    Uninstalling tenacity-8.2.2:
      Successfully uninstalled tenacity-8.2.2
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.30.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.4 which is incompatible.
streamlit 1.30.0 requires tenacity<9,>=8.1.0, but you have tenacity 9.1.2 which is incompatible.


In [ ]:
# google colaboratoryでは正常に動作したので、環境に問題あり。

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

#1 Documentクラスオブジェクトの作成
documents = [
    Document(
        page_content="""\
セダムはベンケイソウ科マンネングザ属で、日本にも自生しているポピュラーな多\
肉植物です。
種類が多くて葉の大きさや形状、カラーバリエーションも豊富なので、組み合わせ\
て寄せ植えにしたり、庭のグランドカバーにしたりして楽しむことができます。
とても丈夫で育てやすく、多肉植物を初めて育てる方にもおすすめです。""",
        metadata={"source": "succulent-plants-doc"},
    ),
    Document(
        page_content="""\
熊童子はベンケイソウ科コチレドン属の多肉植物です。
葉に丸みや厚みがあり、先端には爪のような突起があることから「熊の手」という\
愛称で人気を集めています。
花はオレンジ色のベル型の花を咲かせることがあります。""",
        metadata={"source": "succulent-plants-doc"},
    ),
    Document(
        page_content="""\
エケベリアはベンケイソウ科エケベリア属の多肉植物で、メキシコなど中南米が原\
産です。
まるで花びらのように広がる肉厚な葉が特徴で、秋には紅葉も楽しめます。
品種が多く、室内でも気軽に育てられるので、人気のある多肉植物です。""",
        metadata={"source": "succulent-plants-doc"},
    ),
    Document(
        page_content="""\
ハオルチアは、春と秋に成長するロゼット形の多肉植物です。
密に重なった葉が放射状に展開し、幾何学的で整った株姿になるのが魅力です。
室内でも育てやすく手頃なサイズの多肉植物です。""",
        metadata={"source": "succulent-plants-doc"},
    ),
]

#2 Chromaデータベースの作成
vectorstore = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(),
)

# データベースからコードを取り出す

In [ ]:
vectorstore.similarity_search("熊童子")

# 抽出されたテキストとキーワードの類似さの確認

In [ ]:
vectorstore.similarity_search_with_score("熊童子")

# 文書検索機能を持つLLM

In [ ]:
from langchain_core.runnables import RunnableLambda

#1 Runnable オブジェクトの作成
retriever = RunnableLambda(vectorstore.similarity_search).bind\
(k=1)
retriever.invoke("熊童子")

In [ ]:
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])
model = ChatOpenAI(model="gpt-4o-mini")

#1 Chainの作成
rag_chain = {"context": retriever,
             "question": RunnablePassthrough()} | prompt | model

result = rag_chain.invoke("熊童子について教えてください。")
print(result.content)